#### LIBRERIAS

In [1]:
import pandas as pd
from datetime import datetime
from datetime import date
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import unicodedata
import math
from itertools import cycle
import xlsxwriter
import pyodbc

import random
from IPython.display import display, HTML

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

Posteriormente, en el informe PEP se deben añadir las operaciones sospechosas relacionadas a individuos PEP.
Los Sujetos Obligados deben registrar cualquier operación en que esté involucrada alguna persona que deba ser calificada como PEP, así como informarla por vía electrónica a esta Unidad a la brevedad posible, cuando se considera que se está en presencia de una operación sospechosa.


#### FUNCIONES

In [2]:
def calcula_dv(rut_base):
    # Convertir el RUT base a una cadena y revertirla
    reversed_digits = list(map(int, str(rut_base)[::-1]))
    # Definir los factores (2..7) que se repiten en ciclo
    factors = list(range(2, 8))
    # Calcular la suma de los productos
    suma = sum(d * f for d, f in zip(reversed_digits, factors * (len(reversed_digits) // len(factors) + 1)))
    # Calcular el dígito verificador
    dv = (-suma) % 11
    return 'K' if dv == 10 else str(dv)

def remove_leading_zeros(rut):
    rut_part, dv = rut.split('-')
    rut_part = rut_part.lstrip('0')
    if not rut_part:
        rut_part = '0'
    return f"{rut_part}-{dv}"

def reemplazar_k(identificador):
    return identificador[:-1:] + identificador[-1].upper()

def separar_rut(numero):
    numero_str = str(numero)
    # Tomar los últimos 1 dígito y el resto
    parte_numerica = numero_str[:-1]
    digito_verificador = numero_str[-1]
    # Formatear el RUT con el guion
    rut_formateado = f"{parte_numerica}-{digito_verificador}"
    return rut_formateado

### SE DEBE MODIFICAR LOS SIGUIENTES DATOS DE INPUT

<table>
  <thead>
    <tr>
      <th>QRY_NAME</th>
      <th>FECHA</th>
      <th>DF_NAME</th>
      <th>DESCRIPCIÓN</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>query_operacion_final</td>
      <td>[]</td>
      <td>df_info_personal_test</td>
      <td>Última operación del cliente</td>
    </tr>
    <tr>
      <td>query_info_personal</td>
      <td>[]</td>
      <td>df_info_personal</td>
      <td>Info personal del cliente</td>
    </tr>
      <tr>
      <td>query_ultima_operacion</td>
      <td>[YYYY0-MM0-DD0] ; [YYYY-MM-DD]</td>
      <td> df_ultima_operacion</td>
      <tdOperaciones dentro del periodoe</td>
    </tr>
    <tr>
      <td>query_CustodiaClientes</td>
      <td>[YYYY-MM-DD]</td>
      <td>df_cartera_detalle</td>
      <td>Cartera Vigente Detallada</td>
    </tr>
    <tr>
      <td>query_SaldoCaja</td>
      <td>[YYYY-MM-DD]</td>
      <td>df_saldo_caja</td>
      <td>Datos Caja</td>
    </tr>
    <tr>
      <td>query_APORET</td>
      <td>[YYYY0-MM0-DD0] ; [YYYY-MM-DD]</td>
      <td>df_APORET_detalle</td>
      <td>Aportes Retiros Detalle</td>
    </tr>
    <tr>
      <td>query_IndicadorCustodia</td>
      <td>[YYYY-MM-DD]</td>
      <td>df_indicador</td>
      <td>Indicador custodia</td>
    </tr>
  </tbody>
</table>


In [3]:
## 'YYYY-MM-DD'
F_i = '2024-10-14'  # Fecha inicial
F = '2024-10-18'  # Fecha 
F_cartera_caja = '2024-10-17' # dia previo si es viernes
## CONSULTA DEL SALDO DE CAJA Y CUSTODIA TIENE QUE SER RESPECTO FECHA CIERRE ANTERIOR

In [4]:
FICHA_PEP = 'FICHA PEP/Ficha PEP 18-10-2024'
NIVEL_PEP = 'Relacionados_PEP'

#### CLIENTES PEP

In [5]:
df_ficha_pep = pd.read_excel(f"{FICHA_PEP}.xlsx")
df_nivel_pep = pd.read_excel(f"{NIVEL_PEP}.xlsx")

In [6]:
df_ficha_pep = df_ficha_pep[['NOMBRE','APELLIDO PATERNO', 'APELLIDO MATERNO', 'Dni']]

In [7]:
df_ficha_pep.loc[:, 'Nombre Cliente'] = df_ficha_pep['NOMBRE'].astype(str) +" "+df_ficha_pep['APELLIDO PATERNO'].astype(str) +" "+df_ficha_pep['APELLIDO MATERNO'].astype(str)
df_ficha_pep['Dni'] = df_ficha_pep['Dni'].apply(separar_rut)
del df_ficha_pep['NOMBRE']
del df_ficha_pep['APELLIDO PATERNO']
del df_ficha_pep['APELLIDO MATERNO']
df_ficha_pep = df_ficha_pep.rename(columns={'Dni': 'IDENTIFICADOR', 'Nombre Cliente': 'NOMBRE CLIENTE'})

#### INFORMACION PEP ADICIONAL

In [8]:
df_nivel_pep['IDENTIFICADOR'] = df_nivel_pep['IDENTIFICADOR'].apply(separar_rut)
df_nivel_pep['RUT PEP RELACIONADO'] = df_nivel_pep['RUT PEP RELACIONADO'].apply(separar_rut)
df_nivel_pep['RUT PEP RELACIONADO'] = df_nivel_pep['RUT PEP RELACIONADO'].apply(reemplazar_k)

#### IDENTIFICADORES PEP DE LA FICHA

In [9]:
identificadores_PEP = df_ficha_pep['IDENTIFICADOR'].tolist()

In [10]:
identificadores = df_ficha_pep['IDENTIFICADOR'].tolist()
# SEPARAR POR COMAS
identificadores_str = ','.join([f"'{id}'" for id in identificadores])
### OBTIENE ULTIMA OPERACION DEL CLIENTE

#### CONSULTAS PARTE 0

In [11]:
query_universal = {

    'query_operacion_final': f"""
WITH OperacionesOrdenadas AS (
    SELECT 
        [NUM_CUENTA],
        [IDENTIFICADOR],
        [NOMBRE_CLIENTE],
        [NOMBRE_ASESOR],
        [FECHA_OPERACION],
        [COD_TIPO_OPERACION],
        [DSC_OPERACION_CONCEPTO],
        [COD_MONEDA_OP],
        [NEMOTECNICO],
        [DSC_INSTRUMENTO],
        [COD_SUB_CLASE_INSTRUMENTO],
        [CANTIDAD],
        [MONTO],
        [MONTO_OPERACION],
        ROW_NUMBER() OVER (PARTITION BY [IDENTIFICADOR] ORDER BY [FECHA_OPERACION] DESC) AS rn
    FROM [Capitaria].[dbo].[OPERACION_CLIENTE_JGG]
    WHERE [IDENTIFICADOR] IN ({identificadores_str})
)
SELECT *
FROM OperacionesOrdenadas
WHERE rn = 1;"""
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_info_personal_test = pd.read_sql(query_universal['query_operacion_final'], conn)

# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\2762942999.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_personal_test = pd.read_sql(query_universal['query_operacion_final'], conn)


#### CONSULTAS PARTE 1

In [12]:
identificadores = df_ficha_pep['IDENTIFICADOR'].tolist()
# SEPARAR POR COMAS
identificadores_str = ','.join([f"'{id}'" for id in identificadores])

query_universal = {

    'query_info_personal': f"""
        SELECT [NUM_CUENTA]
              ,[DSC_CUENTA]
              ,[ESTADO_CUENTA]
              ,[COD_MONEDA]
              ,[FECHA_OPERATIVA]
              ,[NOMBRE_CLIENTE]
              ,[NOMBRE_ASESOR]
              ,[FLG_MOV_DESCUBIERTOS]
              ,[DSC_PERFIL_RIESGO]
              ,[NOMBRE_CLIENTE_P]
              ,[FECHA_NACIMIENTO]
              ,[IDENTIFICADOR]
              ,[COD_PAIS]
              ,[PROFESION]
              ,[EMPLEADOR]
              ,[CARGO]
          FROM [Capitaria].[dbo].[CLIENTES_JG]

        WHERE 
        [IDENTIFICADOR] IN ({identificadores_str}); 
    """,

    'query_ultima_operacion': f"""

            SELECT 
                [NUM_CUENTA],
                [IDENTIFICADOR],
                [NOMBRE_CLIENTE],
                [NOMBRE_ASESOR],
                [FECHA_OPERACION],
                [COD_TIPO_OPERACION],
                [DSC_OPERACION_CONCEPTO],
                [COD_MONEDA_OP],
                [NEMOTECNICO],
                [DSC_INSTRUMENTO],
                [COD_SUB_CLASE_INSTRUMENTO],
                [CANTIDAD],
                [MONTO],
                [MONTO_OPERACION]
            FROM [Capitaria].[dbo].[OPERACION_CLIENTE_JGG]
            WHERE [IDENTIFICADOR] IN ({identificadores_str})
            AND FECHA_OPERACION BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
    """
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_info_personal = pd.read_sql(query_universal['query_info_personal'], conn)
df_ultima_operacion = pd.read_sql(query_universal['query_ultima_operacion'], conn)
# CLOSE
conn.close()

## qry ultima operacion toma todas las operaciones del periodo

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\386275727.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_personal = pd.read_sql(query_universal['query_info_personal'], conn)
C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\386275727.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ultima_operacion = pd.read_sql(query_universal['query_ultima_operacion'], conn)


#### CONSULTAS PARTE 3

In [13]:
query_universal = {
    'query_CustodiaClientes': f"""
        SELECT
            C.PRECIO_TASA_MERCADO,
            C.PRECIO_TASA_COMPRA,
            V.NUM_CUENTA,
            V.ID_CUENTA,
            V.DSC_CUENTA AS NOMBRE_CLI,
            C.CANTIDAD AS LIBRE,
            C.GARANTIA,
            C.COMPRAS_PLAZO,
            C.VENTAS_PLAZO,
            CASE 
                WHEN PRESTAMOS IS NULL THEN 0 
                ELSE PRESTAMOS 
            END AS PRESTAMOS_ACC,
            V.IDENTIFICADOR,
            I.NEMOTECNICO,
            V.NOMBRE_ASESOR,
            CASE 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA = 'CLP' THEN CANTIDAD * PRECIO_TASA_MERCADO 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA <> 'CLP' THEN (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * CANTIDAD)
                ELSE (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * CANTIDAD) 
            END AS LIBRE_CLP,
            CASE 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA = 'CLP' THEN GARANTIA * PRECIO_TASA_MERCADO 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA <> 'CLP' THEN (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * GARANTIA)
                ELSE (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * GARANTIA) 
            END AS GARANTIA_CLP,
            C.COMPRAS_PLAZO * C.PRECIO_TASA_MERCADO AS SIM_COMPRA_CLP,
            C.VENTAS_PLAZO * C.PRECIO_TASA_MERCADO AS SIM_VENTA_CLP,
            C.VALOR_MERCADO_CLP,
            I.COD_SUB_CLASE_INSTRUMENTO
        FROM
            dbo.cierre_cartera_resumida AS C WITH (NOLOCK)
            INNER JOIN dbo.VIEW_CUENTAS AS V WITH (NOLOCK) ON C.ID_CUENTA = V.ID_CUENTA
            INNER JOIN dbo.INSTRUMENTO AS I WITH (NOLOCK) ON C.ID_INSTRUMENTO = I.ID_INSTRUMENTO
        WHERE
            C.FECHA_CIERRE = CONVERT(datetime, '{F_cartera_caja}', 120) 
            AND V.IDENTIFICADOR IN ({identificadores_str});
    """
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_cartera_detalle = pd.read_sql(query_universal['query_CustodiaClientes'], conn)
# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\2317256833.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cartera_detalle = pd.read_sql(query_universal['query_CustodiaClientes'], conn)


#### CONSULTAS PARTE 4

In [14]:
query_universal = {
    
    'query_IndicadorCustodia': f"""
        SELECT
            SUM(C.VALOR_MERCADO_CLP) AS SUM_VALOR_MERCADO_CLP
        FROM
            dbo.cierre_cartera_resumida AS C WITH (NOLOCK)
            INNER JOIN dbo.VIEW_CUENTAS AS V WITH (NOLOCK) ON C.ID_CUENTA = V.ID_CUENTA
            INNER JOIN dbo.INSTRUMENTO AS I WITH (NOLOCK) ON C.ID_INSTRUMENTO = I.ID_INSTRUMENTO
        WHERE
            C.FECHA_CIERRE = CONVERT(datetime, '{F_cartera_caja}', 120) 
            AND I.COD_SUB_CLASE_INSTRUMENTO <> 'ACC_INT'
    """

}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_indicador = pd.read_sql(query_universal['query_IndicadorCustodia'], conn)
# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\3363267685.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_indicador = pd.read_sql(query_universal['query_IndicadorCustodia'], conn)


#### CONSULTAS PARTE 5

In [15]:
query_universal = {

    'query_SaldoCaja': f"""

    SELECT DISTINCT 
        A.NOMBRE_ASESOR, 
        A.IDENTIFICADOR,
        b.NUM_CUENTA, 
        b.DSC_CUENTA AS NOMBRE_CLI, 
        b.TIPO_CAJA, 
        b.MONTO_MON_CAJA, 
        b.MONTO_EN_PESOS, 
        b.TRANSITO
        FROM 
        dbo.VIEW_CUENTAS AS A
        INNER JOIN (
        SELECT DISTINCT 
            C.ID_CUENTA, 
            C.NUM_CUENTA, 
            C.DSC_CUENTA, 
            A.FECHA_CIERRE, 
            A.EST_SALDO_CAJA AS ESTADO, 
            CASE 
                WHEN DSC_CAJA_CUENTA = 'CAJA PESOS' THEN 'CAJA PESO' 
                ELSE DSC_CAJA_CUENTA 
            END AS TIPO_CAJA, 
            A.MONTO_MON_CAJA, 
            A.MONTO_MON_CUENTA AS MONTO_EN_PESOS, 
            A.MONTO_X_PAGAR_MON_CTA, 
            A.MONTO_X_COBRAR_MON_CTA, 
            A.MONTO_X_COBRAR_MON_CTA - A.MONTO_X_PAGAR_MON_CTA AS TRANSITO
        FROM 
            dbo.CUENTA AS C WITH (NOLOCK)
        INNER JOIN 
            dbo.CAJA_CUENTA AS B WITH (NOLOCK) ON C.ID_CUENTA = B.ID_CUENTA
        INNER JOIN 
            dbo.SALDO_CAJA AS A WITH (NOLOCK) ON B.ID_CAJA_CUENTA = A.ID_CAJA_CUENTA
        WHERE 
            a.FECHA_CIERRE = CONVERT(datetime, '{F_cartera_caja}', 120) 
            AND A.EST_SALDO_CAJA = 'VIG' 
            AND (A.MONTO_X_PAGAR_MON_CTA + A.MONTO_X_COBRAR_MON_CTA + A.MONTO_MON_CUENTA) <> 0
        ) AS b ON A.NUM_CUENTA = b.NUM_CUENTA
        WHERE 
        A.IDENTIFICADOR IN ({identificadores_str}); 
    """,


        'query_APORET': f"""
        SELECT 
        CARGO_ABONO,
        NUM_CUENTA,	
        IDENTIFICADOR,	
        NOMBRE_CLI,	
        COD_MOV,	
        DSC_MOV_CAJA,	
        FECHA_MOVIMIENTO,	
        MONTO,
        NOMBRE_ASESOR,	
        TIPO_CAJA,	
        COD_MONEDA,		
        T_C,	
        MONTO_CLP

        FROM [Capitaria].[dbo].[MOV_CAJA_CLI_JGG]
        WHERE COD_MOV IN (
            'APO_PAT',
            'APO_PAT_AT',
            'APO_PAT_BT',
            'APO_PAT_LI',
            'APO_PAT_RC',
            'RET_PAT',
            'RET_PAT_BA',
            'RET_PAT_BT',
            'RET_PAT_EX',
            'RET_PAT_LI'
            )
        AND FECHA_MOVIMIENTO BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
        AND IDENTIFICADOR IN ({identificadores_str});
        """
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_saldo_caja = pd.read_sql(query_universal['query_SaldoCaja'], conn)
df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)
# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\2274696490.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_saldo_caja = pd.read_sql(query_universal['query_SaldoCaja'], conn)
C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_18144\2274696490.py:86: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)


In [84]:
id = '10159793-8'
display(df_info_personal[df_info_personal['IDENTIFICADOR'] == id])
s = df_info_personal[df_info_personal['IDENTIFICADOR'] == id]['NOMBRE_CLIENTE']
s_upper = s.str.upper()  # Use .str.upper() to apply upper() to each element in the Series
print(s_upper)


,NUM_CUENTA,DSC_CUENTA,ESTADO_CUENTA,COD_MONEDA,FECHA_OPERATIVA,NOMBRE_CLIENTE,NOMBRE_ASESOR,FLG_MOV_DESCUBIERTOS,DSC_PERFIL_RIESGO,NOMBRE_CLIENTE_P,FECHA_NACIMIENTO,IDENTIFICADOR,COD_PAIS,PROFESION,EMPLEADOR,CARGO
236,10159793/60,VICTOR ANTONIO CONCHA LABRA,HABILITADO,CLP,2024-09-27,Victor Antonio Concha Labra,RACIONAL .,S,CONSERVADOR,Victor Antonio,1965-06-05,10159793-8,CL,Contador,Servicios Integrales,Técnico


236    VICTOR ANTONIO CONCHA LABRA
Name: NOMBRE_CLIENTE, dtype: object


#### DF DESCARGADOS DE CONSULTAS

In [42]:
### BORRAR COLUMNAS DE INFO PERSONAL(CONTIENE ULTIMA OPERACION)
del df_info_personal_test['rn']

KeyError: 'rn'

#### CARTERA DETALLE Y RESUMIDA

In [53]:
df_APORET_detalle = df_APORET_detalle.sort_values(by='MONTO_CLP', ascending = False)

KeyError: 'MONTO_CLP'

In [17]:
### DETALLE CARTERA
columns_q1 = ['NUM_CUENTA', 'NOMBRE_CLI', 'IDENTIFICADOR', 'NOMBRE_ASESOR','NEMOTECNICO','COD_SUB_CLASE_INSTRUMENTO',
              'VALOR_MERCADO_CLP','LIBRE', 'GARANTIA', 'PRECIO_TASA_COMPRA', 'PRECIO_TASA_MERCADO','COMPRAS_PLAZO','VENTAS_PLAZO',
              'PRESTAMOS_ACC','LIBRE_CLP','GARANTIA_CLP','SIM_COMPRA_CLP','SIM_VENTA_CLP']

df_cartera_detalle = df_cartera_detalle[columns_q1]

In [18]:
### RESUMIDA 
df_cartera_RESUMIDA = df_cartera_detalle.pivot_table(
    index=['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR'],
    columns='COD_SUB_CLASE_INSTRUMENTO',
    values='VALOR_MERCADO_CLP',
    aggfunc='sum'
).fillna(0).reset_index()

df_cartera_RESUMIDA.columns.name = None  # ELIMINAR NOMBRE DE COLUMNAS
df_cartera_RESUMIDA['TOTAL CUSTODIA CLP'] = df_cartera_RESUMIDA[[col for col in df_cartera_RESUMIDA.columns if col not in ['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR']]].sum(axis=1)

In [19]:
## SUMAR INDEPENDIENTE DEL ASESOR
cols_no_sumar = ['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR']
cols_a_sumar = [col for col in df_cartera_RESUMIDA.columns if col not in cols_no_sumar]
suma_por_identificador = df_cartera_RESUMIDA.groupby('IDENTIFICADOR')[cols_a_sumar].sum()

df_cartera_RESUMIDA = suma_por_identificador.reset_index()
df_cartera_RESUMIDA = df_cartera_RESUMIDA.sort_values(by='TOTAL CUSTODIA CLP', ascending=False)

In [20]:
## PARA PONER ASESORES ENROLAMIENTO
df_info_personal_ASE = df_info_personal.groupby('IDENTIFICADOR')['NOMBRE_ASESOR'].unique().reset_index()
df_info_personal_ASE.columns = ['IDENTIFICADOR', 'ASESORES_CUSTODIA']

df_info_personal_ASE = df_info_personal.groupby('IDENTIFICADOR').agg({
    'NOMBRE_ASESOR': lambda x: list(x.unique()),
    'PROFESION': 'first',
    'EMPLEADOR': 'first',
    'CARGO': 'first',
    'FECHA_OPERATIVA': 'first',
    'DSC_PERFIL_RIESGO': 'first',
    'FECHA_NACIMIENTO': 'first',
    'COD_PAIS': 'first',
}).fillna(' ').reset_index()

df_info_personal_ASE['IDENTIFICADOR'] = df_info_personal_ASE['IDENTIFICADOR'].apply(reemplazar_k)

#### SALDO CAJA

In [21]:
neworder =  ['NUM_CUENTA','NOMBRE_CLI','IDENTIFICADOR','NOMBRE_ASESOR',
    'TIPO_CAJA', 'MONTO_MON_CAJA', 'MONTO_EN_PESOS', 'TRANSITO']
df_saldo_caja=df_saldo_caja.reindex(columns=neworder)

In [22]:
df_saldo_caja_RESUMIDA = df_saldo_caja.pivot_table(
    index=['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR'],
    columns='TIPO_CAJA',
    values='MONTO_EN_PESOS',
    aggfunc='sum'
).fillna(0).reset_index()

df_saldo_caja_RESUMIDA.columns.name = None
df_saldo_caja_RESUMIDA['TOTAL CAJA CLP'] = df_saldo_caja_RESUMIDA[
    [col for col in df_saldo_caja_RESUMIDA.columns if col not in ['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR']]
].sum(axis=1)

## SUMAR INDEPENDIENTE DEL ASESOR
cols_no_sumar = ['IDENTIFICADOR', 'NOMBRE_CLI', 'NOMBRE_ASESOR']
cols_a_sumar = [col for col in df_saldo_caja_RESUMIDA.columns if col not in cols_no_sumar]
suma_por_identificador = df_saldo_caja_RESUMIDA.groupby('IDENTIFICADOR')[cols_a_sumar].sum()

df_saldo_caja_RESUMIDA = suma_por_identificador.reset_index()
df_saldo_caja_RESUMIDA = df_saldo_caja_RESUMIDA.sort_values(by='TOTAL CAJA CLP', ascending=False)

#### APORTES RETIROS

In [23]:
#del df_APORET_detalle['Unnamed: 0'] esta no estaba
df_APORET_RESUMEN = df_APORET_detalle.groupby(['IDENTIFICADOR','CARGO_ABONO', 'TIPO_CAJA'], as_index=False)['MONTO_CLP'].sum().fillna(' ')

#### RESUMEN

In [24]:
df_cartera_RESUMIDA['IDENTIFICADOR'] = df_cartera_RESUMIDA['IDENTIFICADOR'].apply(reemplazar_k)
df_saldo_caja_RESUMIDA['IDENTIFICADOR'] = df_saldo_caja_RESUMIDA['IDENTIFICADOR'].apply(reemplazar_k)

In [25]:
df_resumen = df_nivel_pep.copy()
df_cartera_RESUMIDA_2 = df_cartera_RESUMIDA.copy()
df_saldo_caja_RESUMIDA_2 = df_saldo_caja_RESUMIDA.copy()


df_resumen = pd.merge(df_resumen, df_cartera_RESUMIDA_2, on=['IDENTIFICADOR'], how='outer')
df_resumen = pd.merge(df_resumen, df_saldo_caja_RESUMIDA_2 , on=['IDENTIFICADOR'], how='outer')

df_resumen = df_resumen.sort_values(by='TOTAL CUSTODIA CLP', ascending=False)

In [47]:
df_info_personal[df_info_personal['IDENTIFICADOR'] == '7972965-5']

,NUM_CUENTA,DSC_CUENTA,ESTADO_CUENTA,COD_MONEDA,FECHA_OPERATIVA,NOMBRE_CLIENTE,NOMBRE_ASESOR,FLG_MOV_DESCUBIERTOS,DSC_PERFIL_RIESGO,NOMBRE_CLIENTE_P,FECHA_NACIMIENTO,IDENTIFICADOR,COD_PAIS,PROFESION,EMPLEADOR,CARGO
238,7972965/50,MAY CHOMALI GARIB,HABILITADO,CLP,2024-10-14,MAY CHOMALI GARIB,BETTERPLAN ADVISORS SPA .,S,ARRIESGADO,MAY,1958-06-18,7972965-5,CL,None,None,None


In [26]:
pivot_df_aporet = df_APORET_RESUMEN.pivot_table(
    index=['IDENTIFICADOR'],
    columns=['CARGO_ABONO', 'TIPO_CAJA'],
    values='MONTO_CLP',
    aggfunc='sum',
).fillna(' ')

# RESETEAR INDICE
pivot_df_aporet = pivot_df_aporet.reset_index()
pivot_df_aporet['IDENTIFICADOR'] = pivot_df_aporet['IDENTIFICADOR'].apply(reemplazar_k)
# AJUSTAR DATA FRAME
pivot_df_aporet.columns = [
    'IDENTIFICADOR', 
    'ABONO_CAJA_DOLAR', 'ABONO_CAJA_PESO', 
    'CARGO_CAJA_DOLAR', 'CARGO_CAJA_PESO']
pivot_df_aporet = pivot_df_aporet.fillna(' ')

In [27]:
pivot_df_aporet_2 = pivot_df_aporet.copy()
df_resumen = pd.merge(df_resumen, pivot_df_aporet_2, on=['IDENTIFICADOR'], how='outer')
df_resumen = pd.merge(df_resumen, df_info_personal_ASE, on=['IDENTIFICADOR'], how='outer')

### AGREGAR ULTIMA OPERACION
df_ultima_operacion['IDENTIFICADOR'] = df_ultima_operacion['IDENTIFICADOR'].apply(reemplazar_k)

In [28]:
df_ultima_operacion_UNIQUE = df_ultima_operacion.groupby('IDENTIFICADOR').agg({
    'COD_SUB_CLASE_INSTRUMENTO': lambda x: list(x.unique())
    }).fillna(' ').reset_index()

In [29]:
df_resumen = pd.merge(df_resumen, df_ultima_operacion_UNIQUE, on=['IDENTIFICADOR'], how='outer')
df_resumen = df_resumen.fillna(' ')

In [30]:
### CANTIDAD DE OPERACIONES
df_cant_operaciones = df_ultima_operacion['IDENTIFICADOR'].value_counts().reset_index(name='CANTIDAD_OPERACIONES')

df_resumen = pd.merge(df_resumen, df_cant_operaciones, on=['IDENTIFICADOR'], how='outer')
df_resumen = df_resumen.fillna(' ')

In [31]:
df_resumen['TOTAL CUSTODIA CLP'] = pd.to_numeric(df_resumen['TOTAL CUSTODIA CLP'], errors='coerce')
df_resumen['ACC_INT'] = pd.to_numeric(df_resumen['ACC_INT'], errors='coerce')

custodia_total = df_indicador['SUM_VALOR_MERCADO_CLP'].values[0]  
custodia_pep_suma = df_resumen['TOTAL CUSTODIA CLP'].sum()-df_resumen['ACC_INT'].sum()

# INDICE COMO PORCENTAJE
indice = (custodia_pep_suma / custodia_total) * 100  # Obtén el índice como número
indice_formateado = f"{indice:.4f}%"  # Formatea el índice con 4 decimales y agrega el símbolo %

# ESTRUCTURA DF INDICADOR
data = {
    'CUSTODIA': ['PEP', 'TOTAL', 'INDICADOR'],
    'MONTO': [custodia_pep_suma, custodia_total, indice_formateado]
}

df_indicador_numero = pd.DataFrame(data)

In [32]:
### RENOMBRAR COLUMNAS
df_resumen = df_resumen.rename(columns={'INFO NAME': 'NOMBRE CLIENTE', 
                                        'NOMBRE_ASESOR': 'ASESORES',
                                        'DSC_PERFIL_RIESGO': 'PERFIL_RIESGO',
                                        'COD_SUB_CLASE_INSTRUMENTO': 'PRODUCTOS_OPERADOS',
                                        'COD_PAIS': 'PAIS'
                                       })

In [33]:
### AÑADE INFO DE ULTIMA OPERACION
df_info_personal_test_2 = df_info_personal_test.copy()
df_info_personal_test_2 = df_info_personal_test_2[['IDENTIFICADOR','FECHA_OPERACION','COD_SUB_CLASE_INSTRUMENTO']]

df_info_personal_test_2['IDENTIFICADOR'] = df_info_personal_test_2['IDENTIFICADOR'].apply(reemplazar_k)

df_resumen = pd.merge(df_resumen, df_info_personal_test_2, on=['IDENTIFICADOR'], how='outer')
df_resumen = df_resumen.fillna(' ')

df_resumen =df_resumen.rename(columns={"COD_SUB_CLASE_INSTRUMENTO": "ULTIMO_INSTRUMENTO_OPERADO", "FECHA_OPERACION": "FECHA_ULTIMA_OPERACION"})

#### INDICADORES

In [34]:
# TOTALES FORMATO NUM
df_resumen['TOTAL CAJA CLP'] = pd.to_numeric(df_resumen['TOTAL CAJA CLP'], errors='coerce')
df_resumen['TOTAL CUSTODIA CLP'] = pd.to_numeric(df_resumen['TOTAL CUSTODIA CLP'], errors='coerce')
# OBTENCION TOP 10
top10_caja = df_resumen.groupby(['IDENTIFICADOR', 'NOMBRE CLIENTE'])['TOTAL CAJA CLP'].sum().nlargest(10).reset_index()
top10_custodia = df_resumen.groupby(['IDENTIFICADOR', 'NOMBRE CLIENTE'])['TOTAL CUSTODIA CLP'].sum().nlargest(10).reset_index()

In [35]:
# CONVERTIR COLUMNAS A NUM
columns_to_convert = ['ACC', 'ACC_INT', 'CFI', 'ETF', 'FMF']
for col in columns_to_convert:
    df_resumen[col] = pd.to_numeric(df_resumen[col], errors='coerce')

# TOP 10 DE CADA PRODUCTO
top10_products = {}
for col in columns_to_convert:
    top10_products[col] = df_resumen.groupby(['IDENTIFICADOR', 'NOMBRE CLIENTE'])[col].sum().nlargest(10).reset_index()


#### CUSTODIA TOP 10 PEP RESPECTO TOTAL PEP

In [36]:
custodia_pep_suma = df_resumen['TOTAL CUSTODIA CLP'].sum()
top10_custodia_suma = top10_custodia['TOTAL CUSTODIA CLP'].sum()

# Calcular el índice como porcentaje
indice_2 = (top10_custodia_suma / custodia_pep_suma) * 100  # Obtén el índice como número
indice_formateado_2 = f"{indice_2:.4f}%"  # Formatea el índice con 4 decimales y agrega el símbolo %

# Crear el DataFrame con la estructura solicitada
data_2 = {
    'CUSTODIA': ['TOP 10 PEP', 'TOTAL PEP', 'INDICADOR'],
    'MONTO': [top10_custodia_suma, custodia_pep_suma, indice_formateado_2]
}

df_indicador_numero2 = pd.DataFrame(data_2)

In [37]:
df_resumen = df_resumen.rename(columns={'INFO NAME': 'NOMBRE CLIENTE', 
                                        'NOMBRE_ASESOR': 'ASESORES',
                                        'DSC_PERFIL_RIESGO': 'PERFIL_RIESGO',
                                        'COD_SUB_CLASE_INSTRUMENTO': 'PRODUCTOS_OPERADOS',
                                        'COD_PAIS': 'PAIS',
                                       })

## GENERAR EXCEL

In [38]:

def format_and_save_excel_with_alternating_colors(filename, logo_path, dfs_with_format=None, dfs_without_format=None, dfs_multiple_on_sheet=None):
    if dfs_with_format is None:
        dfs_with_format = {}
    if dfs_without_format is None:
        dfs_without_format = {}
    if dfs_multiple_on_sheet is None:
        dfs_multiple_on_sheet = {}

    todays_date = date.today().strftime('%Y-%d-%m')

    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        # FORMATEO COLORES ALTERNOS
        def format_sheet_with_alternating_colors(df, sheet_name):
            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=4, header=False)  # Start from row 4

            workbook = writer.book
            worksheet = writer.sheets[sheet_name]

            # Insert the logo at B1 (row 1, column 2)
            worksheet.insert_image('A1', logo_path)

            # Insert today's date in D1
            worksheet.write('C2', todays_date, workbook.add_format({'bold': True, 'font_size': 12}))

            # FORMATO HEADER
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#022596',
                'font_color': 'white',
                'border': 1
            })

            worksheet.set_default_row(15)
            for col_idx in range(len(df.columns)):
                worksheet.set_column(col_idx, col_idx, 20)

            worksheet.add_table(3, 0, len(df) + 3, len(df.columns) - 1, {  # Adjust to row 4 (3-based index)
                'columns': [{'header': col} for col in df.columns],
                'style': 'none'
            })

            # Write the headers manually
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(3, col_num, value, header_format)  # Header row at row 4 (3-based index)

            # Alternate row colors
            format1 = workbook.add_format({'bg_color': '#FFFFFF', 'align': 'left'})
            format2 = workbook.add_format({'bg_color': '#E0FFFF', 'align': 'left'})
            date_format1 = workbook.add_format({'bg_color': '#FFFFFF', 'align': 'left', 'num_format': 'yyyy-mm-dd'})
            date_format2 = workbook.add_format({'bg_color': '#E0FFFF', 'align': 'left', 'num_format': 'yyyy-mm-dd'})

            previous_identifier = None
            format_to_use = format1
            date_format_to_use = date_format1

            for row_num, identifier in enumerate(df['IDENTIFICADOR'], start=4):  # Start at row 4
                if identifier != previous_identifier:
                    format_to_use = format2 if format_to_use == format1 else format1
                    date_format_to_use = date_format2 if date_format_to_use == date_format1 else date_format1
                    previous_identifier = identifier

                for col_num in range(len(df.columns)):
                    value = df.iloc[row_num - 4, col_num]  # Adjust for the new row index

                    if pd.isna(value) or (isinstance(value, float) and (value == float('inf') or value == -float('inf'))):
                        value = ''  # Replace NaN or Infinity with an empty string
                    elif isinstance(value, (list, np.ndarray)):
                        value = ', '.join(map(str, value))  # Convert lists or arrays to a string

                    if df.columns[col_num].startswith('FECHA'):
                        worksheet.write(row_num, col_num, value, date_format_to_use)
                    else:
                        worksheet.write(row_num, col_num, value, format_to_use)

        # FORMATEO HOJA SIN COLORES ALTERNOS
        def format_sheet_without_alternating_colors(df, sheet_name, start_row=3, insert_logo=True):
            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row, header=False)

            workbook = writer.book
            worksheet = writer.sheets[sheet_name]

            # Only insert the logo and date if insert_logo is True
            if insert_logo:
                worksheet.insert_image('A1', logo_path)
                worksheet.write('C2', todays_date, workbook.add_format({'bold': True, 'font_size': 12}))

            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#022596',
                'font_color': 'white',
                'border': 1
            })

            worksheet.set_default_row(15)
            for col_idx in range(len(df.columns)):
                worksheet.set_column(col_idx, col_idx, 20)

            worksheet.add_table(start_row, 0, start_row + len(df), len(df.columns) - 1, {
                'columns': [{'header': col} for col in df.columns],
                'style': 'none'
            })

            # Write the headers manually
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(start_row, col_num, value, header_format)

            cell_format = workbook.add_format({'align': 'left', 'bg_color': '#FFFFFF'})
            date_format = workbook.add_format({'num_format': 'yyyy-mm-dd', 'bg_color': '#FFFFFF'})

            for row_num in range(start_row + 1, start_row + len(df) + 1):
                for col_num in range(len(df.columns)):
                    value = df.iloc[row_num - start_row - 1, col_num]

                    if isinstance(value, (list, np.ndarray)):
                        value = ', '.join(map(str, value))  # Convert lists or arrays to a string
                    elif pd.isna(value) or (isinstance(value, float) and (value == float('inf') or value == -float('inf'))):
                        value = ''  # Replace NaN or Infinity with an empty string

                    if df.columns[col_num].startswith('FECHA'):
                        worksheet.write(row_num, col_num, value, date_format)
                    else:
                        worksheet.write(row_num, col_num, value, cell_format)

        # VARIOS DF EN UNA SHEET
        def format_multiple_dfs_on_sheet(dfs, sheet_name):
            workbook = writer.book
            worksheet = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = worksheet

            # Insert the logo at B1 (row 1, column 2) - only once
            worksheet.insert_image('A1', logo_path)

            # Insert today's date in C1 - only once
            worksheet.write('C2', todays_date, workbook.add_format({'bold': True, 'font_size': 12}))


            current_row = 4  # Start at row 4 after the logo and date
            

            
            for title, df in dfs:
                worksheet.write(current_row - 1, 0, title, workbook.add_format({'bold': True, 'font_color': 'black'}))
                # Call a modified version of the format function that doesn't insert the logo again
                format_sheet_without_alternating_colors(df, sheet_name, start_row=current_row, insert_logo=False)
                current_row += len(df) + 3  # Add space between tables

        # Apply formatting for DataFrames with alternating colors
        for sheet_name, df in (dfs_with_format or {}).items():
            format_sheet_with_alternating_colors(df, sheet_name)

        # Apply formatting for DataFrames without alternating colors
        for sheet_name, df in (dfs_without_format or {}).items():
            format_sheet_without_alternating_colors(df, sheet_name)

        # Apply formatting for multiple DataFrames on one sheet
        for sheet_name, dfs in (dfs_multiple_on_sheet or {}).items():
            format_multiple_dfs_on_sheet(dfs, sheet_name)


In [39]:
from PIL import Image

# Load the logo image
logo_path = 'logo_vector.png'
logo = Image.open(logo_path)

# Resize the image
resized_logo = logo.resize((240, 60))

# Save the resized image to a new file
resized_logo_path = 'resized_logo_vector.png'
resized_logo.save(resized_logo_path)

In [40]:
df_cartera_detalle.rename(columns={
        'NUM_CUENTA': 'Número Cuenta',
        'NOMBRE_CLI': 'Descripción',
        'NOMBRE_ASESOR': 'Asesor',
        'NEMOTECNICO': 'Nemotécnico',
        'COD_SUB_CLASE_INSTRUMENTO': 'Código instrumento',
        'VALOR_MERCADO_CLP': 'Valor Mercado CLP',
        'LIBRE': 'Libre',
        'GARANTIA': 'Garantía',
        'PRECIO_TASA_COMPRA':'Precio tasa compra',
        'PRECIO_TASA_MERCADO':'Precio tasa mercado',
        'COMPRAS_PLAZO': 'Compras Plazo',
        'VENTAS_PLAZO': 'Ventas Plazo',
        'PRESTAMOS_ACC': 'Préstamos',
        'LIBRE_CLP': 'Libre CLP',
        'GARANTIA_CLP': 'Garantía CLP',
        'SIM_COMPRA_CLP': 'Sim compra clp',
        'SIM_VENTA_CLP': 'Sim venta clp'},
inplace=True)



df_resumen.rename(columns={'NOMBRE CLIENTE': 'Descripción',
                           'NIVEL': 'Directo/Indirecto', 
                           'PEP RELACIONADO': 'Pep relacionado', 
                           'POSICION PEP RELACIONADO': 'Posición pep relacionado', 
                           'RUT PEP RELACIONADO': 'Rut pep relacionado', 
                           'RELACION': 'Relación',
                           'ACC': 'Acciones', 
                           'ACC_INT': 'Acciones int', 
                           'CFI': 'CFI', 
                           'ETF': 'ETF', 
                           'FMF': 'FMF', 
                           'TOTAL CUSTODIA CLP': 'Total custodia', 
                           
                           'CAJA DOLAR': 'Caja dolar', 
                           'CAJA PESO': 'Caja peso',
                           'TOTAL CAJA CLP': 'Caja', 
                           
                           'ABONO_CAJA_DOLAR': 'Abono caja dolar', 
                           'ABONO_CAJA_PESO': 'Abono caja peso', 
                           'CARGO_CAJA_DOLAR': 'Cargo caja dolar', 
                           'CARGO_CAJA_PESO': 'Cargo caja peso', 
                           'ASESORES': 'Asesores', 

                           'PROFESION': 'Profesión', 
                           'EMPLEADOR': 'Empleador', 
                           'CARGO': 'Cargo', 
                           'FECHA_OPERATIVA': 'Fecha inicio', 
                           'PERFIL_RIESGO': 'Perfil inversionista', 
                           'FECHA_NACIMIENTO': 'Fecha nacimiento', 
                           'PAIS': 'País', 
                           'PRODUCTOS_OPERADOS': 'Productos operados', 
                           'CANTIDAD_OPERACIONES': 'Número de operaciones', 
                           'FECHA_ULTIMA_OPERACION': 'Última operación', 
                           'ULTIMO_INSTRUMENTO_OPERADO': 'Último instrumento operado'}, 
                  
                  inplace=True)


df_saldo_caja.rename(columns={
            'NUM_CUENTA': 'Número cuenta',
            'NOMBRE_CLI': 'Descripción',
            'NOMBRE_ASESOR': 'Asesor',
            'TIPO_CAJA': 'Tipo caja',
            'MONTO_MON_CAJA': 'Monto caja',
            'MONTO_EN_PESOS': 'Monto en pesos',
            'TRANSITO': 'Tránsito'},

            inplace=True)


df_APORET_detalle.rename(columns={
            'CARGO_ABONO': 'Cargo/Abono',
            'NUM_CUENTA': 'Número cuenta',
            'NOMBRE_CLI': 'Descripción',
            'COD_MOV': 'Código movimiento',
            'DSC_MOV_CAJA': 'Descripción movimiento caja',
            'FECHA_MOVIMIENTO': 'Fecha movimiento',
            'MONTO': 'Monto',
            'NOMBRE_ASESOR': 'Asesor',
            'TIPO_CAJA': 'Tipo caja',
            'COD_MONEDA': 'Código moneda',
            'T_C': 'Tipo Cambio',
            'MONTO_CLP': 'Monto en CLP'},
inplace=True)


df_ultima_operacion.rename(columns={
            'NUM_CUENTA': 'Número cuenta',
            'NOMBRE_CLIENTE': 'Descripción',
            'NOMBRE_ASESOR': 'Asesor',
            'FECHA_OPERACION': 'Fecha operación',
            'COD_TIPO_OPERACION': 'Código tipo operación',
            'DSC_OPERACION_CONCEPTO': 'Descripción operación',
            'COD_MONEDA_OP': 'Código moneda',
            'NEMOTECNICO': 'Nemotécnico',
            'DSC_INSTRUMENTO': 'Descripción instrumento',
            'COD_SUB_CLASE_INSTRUMENTO': 'Código sub clase instrumento',
            'CANTIDAD': 'Cantidad',
            'MONTO': 'Monto',
            'MONTO_OPERACION': 'Monto operación'},
inplace=True)


df_indicador_numero.rename(columns={
            'CUSTODIA': 'Custodia',
            'MONTO': 'Monto'},
inplace=True)

df_indicador_numero2.rename(columns={
            'CUSTODIA': 'Custodia',
            'MONTO': 'Monto'},
inplace=True)

top10_custodia.rename(columns={
            'IDENTIFICADOR': 'Rut',
            'NOMBRE CLIENTE': 'Descripción',
            'TOTAL CUSTODIA CLP': 'Total Custodia CLP'},
inplace=True)

top10_caja.rename(columns={
            'IDENTIFICADOR': 'Rut',
            'NOMBRE CLIENTE': 'Descripción',
            'TOTAL CUSTODIA CLP': 'Total Custodia CLP'},
inplace=True)



################################
df_info_personal_test.rename(columns={
'NUM_CUENTA': 'Número cuenta',
'NOMBRE_CLIENTE': 'Descripción',
'NOMBRE_ASESOR': 'Asesor',
'FECHA_OPERACION': 'Fecha operación',
'COD_TIPO_OPERACION': 'Código tipo operación',
'DSC_OPERACION_CONCEPTO': 'Descripción operación concepto',
'COD_MONEDA_OP': 'Código moneda operación',
'NEMOTECNICO': 'Nematécnico',
'DSC_INSTRUMENTO': 'Descripción instrumento',
'COD_SUB_CLASE_INSTRUMENTO': 'Código sub clase instrumento',
'CANTIDAD': 'Cantidad',
'MONTO': 'Monto',
'MONTO_OPERACION': 'Monto operación'},

inplace=True)


In [41]:

format_and_save_excel_with_alternating_colors(
    'Informe PEP 18-10.xlsx',
    'resized_logo_vector.png',
    dfs_with_format={'Detalle Cartera': df_cartera_detalle,
                     'Detalle Caja': df_saldo_caja, 
                     'Detalle Aportes Retiros': df_APORET_detalle,
                     'Detalle Operaciones': df_ultima_operacion
                    
                    },
    dfs_without_format={
                       'Detalle Ultima Operacion': df_info_personal_test,
                       'Resumen': df_resumen
                       },
    dfs_multiple_on_sheet={
        'Indicadores': [
            ('Custodia PEP respecto total clientes', df_indicador_numero),
            ('Custodia Top 10 respecto total clientes PEP', df_indicador_numero2),
            ('Top 10 Custodia', top10_custodia),
            ('Top 10 Caja', top10_caja)
        ] + [(f'Top 10 {k}', df) for k, df in top10_products.items()]
    }
)

#### FORMAT EXCEL OTHER VERSIONS